## Collaborative Filtering

In [55]:
# import pandas
import pandas as pd
from sklearn.neighbors import NearestNeighbors

We will create an recommender engine based on Item Based Collaborative Filtering (IBCF) which searches for the most similar books based on the user ratings. We can download the data from [here](https://drive.google.com/file/d/1WvTmAfO09TCX7xp7uu06__ziic7JnrL5/view?usp=sharing).

In [56]:
path = 'BX-CSV-Dump/'
book_ratings = pd.read_csv(path +'BX-Book-Ratings.csv',sep=";", encoding="latin")
books = pd.read_csv(path +'BX-Books.csv',sep=";", encoding="latin", error_bad_lines=False)

C:\Users\jared\AppData\Local\Temp/ipykernel_18264/3952556977.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv(path +'BX-Books.csv',sep=";", encoding="latin", error_bad_lines=False)
Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fields, sa

* Explore both datasets

In [57]:
book_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [58]:
book_ratings = book_ratings.sample(10000, random_state=123)

In [59]:
book_ratings.shape

(10000, 3)

In [60]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [61]:
books[books['ISBN'] == '0155061224']

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
225816,0155061224,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...


* create dataframe with name 'df_book_features' from book_ratings that have `ISBN` as index, `User-ID` as columns and values are `Book-Rating`.
    - The data are quite big so it's OK to use a sample only in case your PC has limited RAM.


In [62]:
df_book_features = pd.DataFrame(book_ratings['Book-Rating'], book_ratings['ISBN'], book_ratings['User-ID'])

In [63]:
df_book_features

User-ID,52917,198711,23768,269566,81050,183088,206534,148744,217740,3363,...,14361,37886,57108,133961,189835,151806,125039,46443,198464,184299
ISBN,,,,,,,,,,,,,,,,,,,,,
1558534202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0394426290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0375412530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0446605484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1586480375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0373702310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0061042927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0689714920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
book_ratings.head()

,User-ID,ISBN,Book-Rating
229535,52917,1558534202,6
817972,198711,0394426290,0
101577,23768,0375412530,0
1123282,269566,0446605484,0
339655,81050,1586480375,7


* create the instance of the NearestNeighbors class

In [65]:
nn = NearestNeighbors()

In [66]:
df_book_features.fillna(0, inplace=True)

* fit the NearestNeighbors using'df_book_features'

In [67]:
nn.fit(df_book_features)

NearestNeighbors()

In [74]:
books[books['ISBN'] == '1558534202']

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
103027,1558534202,Life's Little Treasure Book On Friendship (Lif...,H. Jackson Brown,1996,Rutledge Hill Press,http://images.amazon.com/images/P/1558534202.0...,http://images.amazon.com/images/P/1558534202.0...,http://images.amazon.com/images/P/1558534202.0...


In [76]:
books[books['ISBN'] == '1558534202']['Book-Title'].values

array(["Life's Little Treasure Book On Friendship (Life's Little Treasure Books (Hardcover))"],
      dtype=object)

In [91]:
df_book_features.loc['1558534202'].ravel()

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

* create function that returns top 5 most similar movies (according to KNN model) for selected ISBN
    * the input will be Book-Title from the DataFrame books 
    * the output will be the Book-Titles of the top 5 most similar books.
    * for every book in the top 5 most similar books, print also the distance from the selected book (ISBN we chose as input to the function)

In [104]:
import numpy as np
def top5(title):
    isbn = books[books['Book-Title'] == title]['ISBN'].iloc[0]
    dist, neighbors = nn.kneighbors([df_book_features.loc[isbn].ravel()], 5, return_distance = True)
    print(dist)
    titles = []
    for neighbor in neighbors:
        titles.append(books[books['ISBN'].isin(neighbor)]['Book-Title'])
    return titles

* Apply the function to book of your choice

In [103]:
top5("Life's Little Treasure Book On Friendship (Life's Little Treasure Books (Hardcover))")

[Series([], Name: Book-Title, dtype: object)]